In [159]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

## Tratamento

In [160]:
# Lendo DF em relação ao mês de Julho
csv = pd.read_csv('dados/precos-gasolina-etanol-07.csv', delimiter=';')
comb_jul = pd.DataFrame(csv)
comb_jul.head(1)

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,SE,ES,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,NaN,LACE,29703-030,GASOLINA ADITIVADA,01/07/2022,"7,48",NaN,R$ / litro,VIBRA ENERGIA


In [161]:
# Lendo DF em relação ao mês de Agosto
csv = pd.read_csv('dados/precos-gasolina-etanol-08.csv', delimiter=';')
comb_ago = pd.DataFrame(csv)
comb_ago.head(1)

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,NE,AL,ARAPIRACA,AUTO POSTO M M GARROTE LTDA,08.738.994/0001-50,RODOVIA AL-220,5848,KM 96,BOM SUCESSO,57309-035,ETANOL,01/08/2022,"4,79",NaN,R$ / litro,VIBRA ENERGIA


In [162]:
# Concatenando os dois DF's
conjunto_dados = pd.concat([comb_jul, comb_ago])
conjunto_dados.head(1)

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira
0,SE,ES,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,NaN,LACE,29703-030,GASOLINA ADITIVADA,01/07/2022,"7,48",NaN,R$ / litro,VIBRA ENERGIA


In [163]:
# Resetando Index no DF concatenado
conjunto_dados.reset_index(0, drop=True, inplace=True)

In [164]:
# Checando a % de valores nulos em cada coluna do DF
((conjunto_dados.isnull().sum() / conjunto_dados.shape[0]) * 100).sort_values(ascending=False)

Valor de Compra      100.000000
Complemento           78.707461
Bairro                 0.323960
Numero Rua             0.055828
Regiao - Sigla         0.000000
Estado - Sigla         0.000000
Municipio              0.000000
Revenda                0.000000
CNPJ da Revenda        0.000000
Nome da Rua            0.000000
Cep                    0.000000
Produto                0.000000
Data da Coleta         0.000000
Valor de Venda         0.000000
Unidade de Medida      0.000000
Bandeira               0.000000
dtype: float64

In [165]:
# Dropando a coluna 'Valor de Compra' pois a mesma contém apenas valores nulos
conjunto_dados.drop('Valor de Compra', axis=1, inplace=True)

In [166]:
# Transformando os valores da coluna 'Valor de Venda' para float
conjunto_dados['Valor de Venda'] = conjunto_dados['Valor de Venda'].apply(lambda y: float(str(y.replace(',','.'))))

In [167]:
# Preenchendo valores nulos da coluna 'Complemento' com 0
conjunto_dados.Complemento.fillna('0', inplace=True)

In [168]:
# Trasnformando as datas da coluna 'Data da Coleta' para datetime
conjunto_dados['Data da Coleta'] = pd.to_datetime(conjunto_dados['Data da Coleta'], format='%d/%m/%Y')
conjunto_dados.head(1)

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Unidade de Medida,Bandeira
0,SE,ES,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,0,LACE,29703-030,GASOLINA ADITIVADA,2022-07-01,7.48,R$ / litro,VIBRA ENERGIA


- Funções

In [169]:
# Definindo função lambda para formatar a saída de valores para estilo monetário
formatar = lambda x: f'R$ {x:.3f}'

In [170]:
# Passando strings para minusculas no DF
to_lower = lambda x: x.lower() if isinstance(x, str) else x

In [171]:
# Criando função que acrecenta coluna de dias da semana
def cria_semanas(df):
    dias_semana = {'Sunday':'Domingo', 'Monday':'Segunda', 'Tuesday':'Terça', 'Wednesday':'Quarta', 'Thursday':'Quinta', 'Friday':'Sexta', 'Saturday':'Sábado'}
    df['Dias da Semana'] = df['Data da Coleta'].apply(lambda x: x.strftime('%A'))
    df['Dias da Semana'] = df['Dias da Semana'].replace(dias_semana)
    return df

In [172]:
# Aplicando a função no DF
conjunto_dados = cria_semanas(conjunto_dados)

In [173]:
# Passando strings para minusculas no DF
conjunto_dados['Municipio'] = conjunto_dados['Municipio'].apply(to_lower)
conjunto_dados['Revenda'] = conjunto_dados['Revenda'].apply(to_lower)
conjunto_dados['Nome da Rua'] = conjunto_dados['Nome da Rua'].apply(to_lower)
conjunto_dados['Complemento'] = conjunto_dados['Complemento'].apply(to_lower)
conjunto_dados['Bairro'] = conjunto_dados['Bairro'].apply(to_lower)
conjunto_dados['Produto'] = conjunto_dados['Produto'].apply(to_lower)
conjunto_dados['Bandeira'] = conjunto_dados['Bandeira'].apply(to_lower)

In [174]:
conjunto_dados.head()

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Unidade de Medida,Bandeira,Dias da Semana
0,SE,ES,colatina,sao francisco de assis comercio de combustivei...,08.519.545/0001-10,praca fidelis ferrari,35,0,lace,29703-030,gasolina aditivada,2022-07-01,7.48,R$ / litro,vibra energia,Sexta
1,SE,ES,colatina,sao francisco de assis comercio de combustivei...,08.519.545/0001-10,praca fidelis ferrari,35,0,lace,29703-030,gasolina,2022-07-01,7.38,R$ / litro,vibra energia,Sexta
2,SE,ES,colatina,posto motocap ltda,04.814.652/0001-10,praca almirante barroso,52,letra a,lace,29703-055,etanol,2022-07-01,5.99,R$ / litro,vibra energia,Sexta
3,SE,ES,colatina,posto motocap ltda,04.814.652/0001-10,praca almirante barroso,52,letra a,lace,29703-055,gasolina,2022-07-01,7.39,R$ / litro,vibra energia,Sexta
4,CO,GO,goiania,justiniano & justiniano ltda,04.489.287/0001-16,avenida jose leandro da cruz,"2,36","quadra 18, lote 01",parque amazonia,74840-390,gasolina aditivada,2022-07-01,6.49,R$ / litro,ipiranga,Sexta


In [175]:
conjunto_dados.dtypes

Regiao - Sigla               object
Estado - Sigla               object
Municipio                    object
Revenda                      object
CNPJ da Revenda              object
Nome da Rua                  object
Numero Rua                   object
Complemento                  object
Bairro                       object
Cep                          object
Produto                      object
Data da Coleta       datetime64[ns]
Valor de Venda              float64
Unidade de Medida            object
Bandeira                     object
Dias da Semana               object
dtype: object

In [176]:
conjunto_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127176 entries, 0 to 127175
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Regiao - Sigla     127176 non-null  object        
 1   Estado - Sigla     127176 non-null  object        
 2   Municipio          127176 non-null  object        
 3   Revenda            127176 non-null  object        
 4   CNPJ da Revenda    127176 non-null  object        
 5   Nome da Rua        127176 non-null  object        
 6   Numero Rua         127105 non-null  object        
 7   Complemento        127176 non-null  object        
 8   Bairro             126764 non-null  object        
 9   Cep                127176 non-null  object        
 10  Produto            127176 non-null  object        
 11  Data da Coleta     127176 non-null  datetime64[ns]
 12  Valor de Venda     127176 non-null  float64       
 13  Unidade de Medida  127176 non-null  object  

## Trabalhando nos dados

In [177]:
#1 Como se comportam o preço dos combustíveis durante os dois meses citados? (Jul/Ago) Os valores do etanol e da gasolina tiveram uma (tendencia?) de queda ou diminuição?

In [178]:
comb_jul = conjunto_dados[(conjunto_dados['Data da Coleta'] < '2022-08-01')] 
comb_ago = conjunto_dados[(conjunto_dados['Data da Coleta'] > '2022-08-01')]

In [179]:
preco_comb_jul = comb_jul.groupby(['Regiao - Sigla']).mean()
preco_comb_jul.columns = ['Preço Médio']
preco_comb_jul['Preço Médio'] = preco_comb_jul['Preço Médio'].apply(formatar)

display(preco_comb_jul.sort_values(by='Preço Médio', ascending=True))

,Preço Médio
Regiao - Sigla,
CO,R$ 5.327
SE,R$ 5.408
S,R$ 5.775
NE,R$ 6.206
N,R$ 6.214


In [180]:
preco_comb_ago = comb_ago.groupby(['Regiao - Sigla']).mean()
preco_comb_ago.columns = ['Preço Médio']
preco_comb_ago['Preço Médio'] = preco_comb_ago['Preço Médio'].apply(formatar)

display(preco_comb_ago.sort_values(by='Preço Médio', ascending=True))

,Preço Médio
Regiao - Sigla,
CO,R$ 4.804
SE,R$ 4.891
S,R$ 5.225
NE,R$ 5.369
N,R$ 5.594


In [181]:
# 2. Qual o preço médio da gasolina e do etanol nesses dois meses?

In [182]:
relacao_comb_preco_jul = comb_jul[['Produto','Valor de Venda', 'Regiao - Sigla']].groupby(['Produto', 'Regiao - Sigla']).mean()
relacao_comb_preco_jul.columns = ['Preço Médio']
relacao_comb_preco_jul['Preço Médio'] = relacao_comb_preco_jul['Preço Médio'].apply(formatar)

display(relacao_comb_preco_jul.sort_values(by='Produto', ascending=True))

Preço Médio
Produto            Regiao - Sigla            
etanol             CO                R$ 4.325
                   N                 R$ 5.637
                   NE                R$ 5.605
                   S                 R$ 5.105
                   SE                R$ 4.364
gasolina           CO                R$ 5.894
                   N                 R$ 6.322
                   NE                R$ 6.443
                   S                 R$ 5.939
                   SE                R$ 5.927
gasolina aditivada CO                R$ 6.034
                   N                 R$ 6.458
                   NE                R$ 6.601
                   S                 R$ 6.079
                   SE                R$ 6.141

In [183]:
relacao_comb_preco_ago = comb_ago[['Produto','Valor de Venda', 'Regiao - Sigla']].groupby(['Produto', 'Regiao - Sigla']).mean()
relacao_comb_preco_ago.columns = ['Preço Médio']
relacao_comb_preco_ago['Preço Médio'] = relacao_comb_preco_ago['Preço Médio'].apply(formatar)

display(relacao_comb_preco_ago.sort_values(by='Produto', ascending=True))

Preço Médio
Produto            Regiao - Sigla            
etanol             CO                R$ 3.912
                   N                 R$ 5.098
                   NE                R$ 4.925
                   S                 R$ 4.612
                   SE                R$ 3.965
gasolina           CO                R$ 5.301
                   N                 R$ 5.681
                   NE                R$ 5.528
                   S                 R$ 5.373
                   SE                R$ 5.343
gasolina aditivada CO                R$ 5.446
                   N                 R$ 5.809
                   NE                R$ 5.681
                   S                 R$ 5.505
                   SE                R$ 5.551

In [212]:
# 3. Quais os 5 estados com o preço médio da gasolina e do etanol mais caros?

# Levando em consideração os 2 meses e os 2 combustíveis juntos,Gasolina(comum/aditiva) e Etanol.
dados_agrupados_estado = conjunto_dados.copy()

media_combs_estados = dados_agrupados_estado.groupby(dados_agrupados_estado['Estado - Sigla'])[['Valor de Venda']].mean()
media_combs_estados.columns = ['Preço Médio']
media_combs_estados['Preço Médio'] = media_combs_estados['Preço Médio'].apply(formatar)

print('OS 5 ESTADOS COM O MAIOR PREÇO MÉDIO DE GASOLINA COMUM/ADITIVADA E ETANOL NO BIMESTRE.')
display(media_combs_estados.sort_values(by='Preço Médio', ascending=False).head())

OS 5 ESTADOS COM O MAIOR PREÇO MÉDIO DE GASOLINA COMUM/ADITIVADA E ETANOL NO BIMESTRE.


,Preço Médio
Estado - Sigla,
RR,R$ 6.021
PA,R$ 5.999
AC,R$ 5.997
CE,R$ 5.868
RN,R$ 5.852


In [213]:
# Fazendo a média bimestral com a Gasolina, Gasolina Aditivada e o Etanol separados.
media_gas_adt_estado = dados_agrupados_estado.loc[dados_agrupados_estado['Produto'] == 'gasolina aditivada'].groupby(dados_agrupados_estado['Estado - Sigla'])

df_media_gas_adt_estado = media_gas_adt_estado[['Valor de Venda']].mean()
df_media_gas_adt_estado.columns = ['Preço Médio']
df_media_gas_adt_estado['Preço Médio'] = df_media_gas_adt_estado['Preço Médio'].apply(formatar)

print('OS 5 ESTADOS COM O MAIOR PREÇO MÉDIO DE GASOLINA ADITIVADA NO BIMESTRE.')
display(df_media_gas_adt_estado.sort_values(by='Preço Médio', ascending=False).head())

OS 5 ESTADOS COM O MAIOR PREÇO MÉDIO DE GASOLINA ADITIVADA NO BIMESTRE.


,Preço Médio
Estado - Sigla,
PI,R$ 6.315
CE,R$ 6.198
AC,R$ 6.178
TO,R$ 6.177
PA,R$ 6.174


In [214]:
media_gas_estado = dados_agrupados_estado.loc[dados_agrupados_estado['Produto'] == 'gasolina'].groupby(dados_agrupados_estado['Estado - Sigla'])
print('OS 5 ESTADOS COM O MAIOR PREÇO MÉDIO DE GASOLINA COMUM NO BIMESTRE.')
df_media_gas_estado = media_gas_estado[['Valor de Venda']].mean()
df_media_gas_estado.columns = ['Preço Médio']
df_media_gas_estado['Preço Médio'] = df_media_gas_estado['Preço Médio'].apply(formatar)

display(df_media_gas_estado.sort_values(by='Preço Médio', ascending=False).head())

OS 5 ESTADOS COM O MAIOR PREÇO MÉDIO DE GASOLINA COMUM NO BIMESTRE.


,Preço Médio
Estado - Sigla,
AC,R$ 6.211
AM,R$ 6.188
PI,R$ 6.075
TO,R$ 6.046
BA,R$ 6.031


In [215]:
print('OS 5 ESTADOS COM O MAIOR PREÇO MÉDIO DE ETANOL NO BIMESTRE.')
media_etanol_estado = dados_agrupados_estado.loc[dados_agrupados_estado['Produto'] == 'etanol'].groupby(dados_agrupados_estado['Estado - Sigla'])
df_media_etanol_estado = media_etanol_estado[['Valor de Venda']].mean()
df_media_etanol_estado.columns = ['Preço Médio']
df_media_etanol_estado['Preço Médio'] = df_media_etanol_estado['Preço Médio'].apply(formatar)

display(df_media_etanol_estado.sort_values(by='Preço Médio', ascending=False).head())

OS 5 ESTADOS COM O MAIOR PREÇO MÉDIO DE ETANOL NO BIMESTRE.


,Preço Médio
Estado - Sigla,
AP,R$ 5.896
RS,R$ 5.660
PA,R$ 5.625
RO,R$ 5.599
RR,R$ 5.568


In [216]:
# 4. Qual o preço médio da gasolina e do etanol por estado?

# Fazendo a média bimestral da Gasolina,da Gasolina Aditivada e do Etanol.
df_media_comb_estado = pd.concat([df_media_gas_adt_estado, df_media_gas_estado, df_media_etanol_estado], axis=1)
df_media_comb_estado.columns = ['Gasolina Aditivada','Gasolina Comum','Etanol']
print('Tabela com a média dos combustíveis por estado:\n')

df_media_comb_estado.style.highlight_max(color='#0C9E02', axis=0).highlight_min(color='#CF0909')

Tabela com a média dos combustíveis por estado:



,Gasolina Aditivada,Gasolina Comum,Etanol
Estado - Sigla,,,
AC,R$ 6.178,R$ 6.211,R$ 5.377
AL,R$ 6.101,R$ 5.945,R$ 5.241
AM,R$ 6.118,R$ 6.188,R$ 4.953
AP,R$ 5.433,R$ 5.107,R$ 5.896
BA,R$ 6.140,R$ 6.031,R$ 5.074
CE,R$ 6.198,R$ 5.955,R$ 5.482
DF,R$ 5.700,R$ 5.601,R$ 4.749
ES,R$ 5.931,R$ 5.796,R$ 4.964
GO,R$ 5.648,R$ 5.519,R$ 4.060


In [223]:
# 5-Qual o município que possui o menor preço para a gasolina e para o etanol?

# primeiro, criamos um dataframe novo agrupando os dados de Produto e Municipio
produto_municipio = conjunto_dados.groupby(["Produto","Municipio"])

In [224]:
# Em seguida, extraimos apenas os valores mínimos, de acordo com a pergunta
min_municipio = produto_municipio[['Valor de Venda']].min()

# Criamos um dataframe contendo apenas os dados que retornavam as querys contendo Etanol, classificamos pelos valor de venda e resetamos o index na primeira linha
min_municipio_etanol = min_municipio.query("Produto == 'etanol'").sort_values(by='Valor de Venda').head(1)
# minmunicipioetanol = minmunicipioetanol.reset_index().head(1)

# Criamos um dataframe contendo apenas os dados que retornavam as querys contendo Gasolina, classificamos pelos valor de venda e resetamos o index na primeira linha
min_municipio_gas = min_municipio.query("Produto == 'gasolina'").sort_values(by='Valor de Venda').head(1)
# minmunicipiogasolina= minmunicipiogasolina.reset_index().head(1)

# Criamos um dataframe contendo apenas os dados que retornavam as querys contendo Gasolina Aditivada, classificamos pelos valor de venda e resetamos o index na primeira linha
min_municipio_gas_adi = min_municipio.query("Produto == 'gasolina aditivada'").sort_values(by='Valor de Venda').head(1)
# minmunicipiogasolinaaditivada = minmunicipiogasolinaaditivada.reset_index().head(1)

# Concatenamos os três arquivos
minimas = pd.concat([min_municipio_etanol, min_municipio_gas, min_municipio_gas_adi])
minimas.columns = ['Menor Preço']
minimas['Menor Preço'] = minimas['Menor Preço'].apply(formatar)
minimas

,,Menor Preço
Produto,Municipio,
etanol,presidente prudente,R$ 2.890
gasolina,passo fundo,R$ 4.130
gasolina aditivada,passo fundo,R$ 4.130


In [225]:
# 6-Qual o município que possui o maior preço para a gasolina e para o etanol?

# extraimos apenas os valores máximos, de acordo com a pergunta
max_municipio = produto_municipio[['Valor de Venda']].max()

# Criamos um dataframe contendo apenas os dados que retornavam as querys contendo Etanol, classificamos pelos valor de venda e resetamos o index na primeira linha
max_municipio_etanol = min_municipio.query("Produto == 'etanol'").sort_values(by='Valor de Venda', ascending=False).head(1)
# maxmunicipioetanol= maxmunicipioetanol.reset_index().head(1)

# Criamos um dataframe contendo apenas os dados que retornavam as querys contendo Gasolina, classificamos pelos valor de venda e resetamos o index na primeira linha
max_municipio_gas = max_municipio.query("Produto == 'gasolina'").sort_values(by='Valor de Venda', ascending=False).head(1)
# maxmunicipiogasolina=maxmunicipiogasolina.reset_index().head(1)

# Criamos um dataframe contendo apenas os dados que retornavam as querys contendo Gasolina Aditivada, classificamos pelos valor de venda e resetamos o index na primeira linha
max_municipio_gas_adi = max_municipio.query("Produto == 'gasolina aditivada'").sort_values(by='Valor de Venda', ascending=False).head(1)
# maxmunicipiogasolinaaditivada=maxmunicipiogasolinaaditivada.reset_index().head(1)

# Concatenamos os três arquivos
maximas = pd.concat([max_municipio_etanol, max_municipio_gas, max_municipio_gas_adi])
maximas.columns = ['Maior Preço']
maximas['Maior Preço'] = maximas['Maior Preço'].apply(formatar)
maximas

,,Maior Preço
Produto,Municipio,
etanol,uruguaiana,R$ 6.190
gasolina,gurupi,R$ 8.750
gasolina aditivada,mage,R$ 9.270


In [194]:
# 7. Qual a região que possui o maior valor médio da gasolina?

# Criando uma cópia do DF original onde temos apenas os registros da Gasolina
df_gas = conjunto_dados.query('Produto == "gasolina"').copy()

# Ordenando o novo dataframe com o valor de venda da gasolina por região
produtos_gas = df_gas.groupby(['Produto', 'Regiao - Sigla'])[['Valor de Venda']].mean()
produtos_gas.columns = ['Maior Valor Médio']
produtos_gas['Maior Valor Médio'] = produtos_gas['Maior Valor Médio'].apply(formatar)

produtos_gas.style.highlight_max(color='#0C9E02')

In [226]:
# Criando uma cópia do DF original onde temos apenas os registros da Gasolina Aditivada
df_gas_adi = conjunto_dados.query('Produto == "gasolina aditivada"').copy()

# Ordenando o novo dataframe com o valor de venda da gasolina aditivada por região
produtos_gas_adi = df_gas_adi.groupby(['Produto', 'Regiao - Sigla'])[['Valor de Venda']].mean()
produtos_gas_adi.columns = ['Maior Valor Médio']
produtos_gas_adi['Maior Valor Médio'] = produtos_gas_adi['Maior Valor Médio'].apply(formatar)

produtos_gas_adi.style.highlight_max(color='#0C9E02')

In [229]:
# 8. Qual a região que possui o menor valor médio do etanol?

# Criando uma cópia do DF original onde temos apenas os registros do Etanol
df_etanol = conjunto_dados.query('Produto == "etanol"').copy()

# Ordenando o novo dataframe com o valor de venda do etanol por região
produtos_etanol = df_etanol.groupby(['Produto', 'Regiao - Sigla'])[['Valor de Venda']].mean()
produtos_etanol.columns = ['Menor Valor Médio']
produtos_etanol['Menor Valor Médio'] = produtos_etanol['Menor Valor Médio'].apply(formatar)

produtos_etanol.style.highlight_min(color='#CF0909')

In [ ]:
#9. Há alguma correlação entre o valor do combustível (gasolina e etanol) e a região onde ele é vendido?
#   Há uma correlação positiva muito forte entre os valores do combustível com base na média por região.

In [230]:
media_combs_regiao = conjunto_dados.groupby(['Regiao - Sigla','Produto'])[['Valor de Venda']].mean()
media_combs_regiao.columns = ['Preço Médio']
media_combs_regiao['Preço Médio'] = media_combs_regiao['Preço Médio'].apply(formatar)

media_combs_regiao

Preço Médio
Regiao - Sigla Produto                       
CO             etanol                R$ 4.107
               gasolina              R$ 5.582
               gasolina aditivada    R$ 5.721
N              etanol                R$ 5.341
               gasolina              R$ 5.972
               gasolina aditivada    R$ 6.106
NE             etanol                R$ 5.246
               gasolina              R$ 5.956
               gasolina aditivada    R$ 6.109
S              etanol                R$ 4.842
               gasolina              R$ 5.641
               gasolina aditivada    R$ 5.778
SE             etanol                R$ 4.150
               gasolina              R$ 5.616
               gasolina aditivada    R$ 5.827

In [231]:
media_regiao_2 = media_combs_regiao.unstack('Regiao - Sigla', 'Produto')
media_regiao_2

# Plotar gráfico multi-barras e explciar a correlação a partir dai

Preço Médio                                        
Regiao - Sigla              CO         N        NE         S        SE
Produto                                                               
etanol                R$ 4.107  R$ 5.341  R$ 5.246  R$ 4.842  R$ 4.150
gasolina              R$ 5.582  R$ 5.972  R$ 5.956  R$ 5.641  R$ 5.616
gasolina aditivada    R$ 5.721  R$ 6.106  R$ 6.109  R$ 5.778  R$ 5.827

In [ ]:
# 10. Há alguma correlação entre o valor do combustível (gasolina e etanol) e a bandeira que vende ele?

In [232]:
media_combs_bandeira = conjunto_dados.groupby(['Bandeira', 'Produto'])[['Valor de Venda']].mean()
media_combs_bandeira

Valor de Venda
Bandeira    Produto                           
alesat      etanol                    4.657715
            gasolina                  5.798276
            gasolina aditivada        5.925192
americanoil etanol                    5.190000
            gasolina                  5.847826
...                                        ...
walendowsky gasolina                  5.852222
            gasolina aditivada        5.862222
watt        etanol                    4.104545
            gasolina                  6.027273
            gasolina aditivada        6.960000

[145 rows x 1 columns]

In [233]:
media_bandeira_1 = media_combs_bandeira.unstack()
media_bandeira_1.head()

# Verificar o que cada bandeira vende, pois isso pode explicar os valores nulos em alguma delas

Valor de Venda                             
Produto             etanol  gasolina gasolina aditivada
Bandeira                                               
alesat            4.657715  5.798276           5.925192
americanoil       5.190000  5.847826           5.915333
atem' s           5.158268  5.974562           5.916859
atlântica         4.942308  5.791750           5.837419
branca            4.373859  5.645761           5.762779

In [234]:
media_bandeira_1.reset_index(inplace=True)
media_bandeira_1.head()

Bandeira Valor de Venda                             
Produto                      etanol  gasolina gasolina aditivada
0             alesat       4.657715  5.798276           5.925192
1        americanoil       5.190000  5.847826           5.915333
2            atem' s       5.158268  5.974562           5.916859
3          atlântica       4.942308  5.791750           5.837419
4             branca       4.373859  5.645761           5.762779

In [235]:
media_bandeira_1.columns

MultiIndex([(      'Bandeira',                   ''),
            ('Valor de Venda',             'etanol'),
            ('Valor de Venda',           'gasolina'),
            ('Valor de Venda', 'gasolina aditivada')],
           names=[None, 'Produto'])

In [236]:
# Criando um groupby dos registros da SUL COMBUSTÍVEIS e mostrando o df
bandeira_sul_comb = conjunto_dados.loc[(conjunto_dados['Bandeira'] == 'sul combustíveis'), :].copy()
bandeira_sul_comb.head()

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Unidade de Medida,Bandeira,Dias da Semana
4801,S,RS,sao gabriel,jp santa lucia comercio de combustiveis ltda,17.695.813/0019-75,rua mario avancini dos santos,34,0,gabrielense,97306-002,gasolina aditivada,2022-07-04,7.14,R$ / litro,sul combustíveis,Segunda
4802,S,RS,sao gabriel,jp santa lucia comercio de combustiveis ltda,17.695.813/0019-75,rua mario avancini dos santos,34,0,gabrielense,97306-002,gasolina,2022-07-04,6.99,R$ / litro,sul combustíveis,Segunda
8313,S,RS,santa maria,treino comercio de combustiveis - eireli,33.517.308/0001-68,rua assis brasil,395,0,itarare,97045-070,gasolina aditivada,2022-07-05,6.49,R$ / litro,sul combustíveis,Terça
8314,S,RS,santa maria,treino comercio de combustiveis - eireli,33.517.308/0001-68,rua assis brasil,395,0,itarare,97045-070,gasolina,2022-07-05,6.39,R$ / litro,sul combustíveis,Terça
18441,S,RS,santa maria,treino comercio de combustiveis - eireli,33.517.308/0001-68,rua assis brasil,395,0,itarare,97045-070,gasolina,2022-07-11,5.83,R$ / litro,sul combustíveis,Segunda


In [237]:
# Conferindo os valores únicos desses registros e confirmando que a SUL COMBUSTÍVEIS 
bandeira_sul_comb['Produto'].value_counts()

gasolina aditivada    20
gasolina              20
Name: Produto, dtype: int64

In [238]:
# Criando um groupby dos registros da RUFF C.J. e mostrando o df
bandeira_ruff_comb = conjunto_dados.loc[(conjunto_dados['Bandeira'] == 'ruff c.j.'), :].copy()
bandeira_ruff_comb.head()

,Regiao - Sigla,Estado - Sigla,Municipio,Revenda,CNPJ da Revenda,Nome da Rua,Numero Rua,Complemento,Bairro,Cep,Produto,Data da Coleta,Valor de Venda,Unidade de Medida,Bandeira,Dias da Semana
26062,SE,SP,sao joao da boa vista,comercial de petroleo 4c ltda,31.819.835/0001-00,rua tiradentes,45,0,rosario,13870-260,gasolina,2022-07-13,5.69,R$ / litro,ruff c.j.,Quarta
26063,SE,SP,sao joao da boa vista,comercial de petroleo 4c ltda,31.819.835/0001-00,rua tiradentes,45,0,rosario,13870-260,etanol,2022-07-13,4.13,R$ / litro,ruff c.j.,Quarta
40685,SE,SP,sao joao da boa vista,comercial de petroleo 4c ltda,31.819.835/0001-00,rua tiradentes,45,0,rosario,13870-260,etanol,2022-07-20,3.99,R$ / litro,ruff c.j.,Quarta
40686,SE,SP,sao joao da boa vista,comercial de petroleo 4c ltda,31.819.835/0001-00,rua tiradentes,45,0,rosario,13870-260,gasolina,2022-07-20,5.69,R$ / litro,ruff c.j.,Quarta
51314,SE,SP,sao joao da boa vista,comercial de petroleo 4c ltda,31.819.835/0001-00,rua tiradentes,45,0,rosario,13870-260,etanol,2022-07-26,3.99,R$ / litro,ruff c.j.,Terça


In [239]:
bandeira_ruff_comb['Produto'].value_counts()

gasolina    6
etanol      6
Name: Produto, dtype: int64

In [240]:
media_bandeira_1['Bandeira'] = media_bandeira_1['Bandeira'].astype('category').cat.codes
media_bandeira_1.corr()

Bandeira Valor de Venda            \
Produto                                             etanol  gasolina   
               Produto                                                 
Bandeira                           1.000000      -0.186898 -0.162000   
Valor de Venda etanol             -0.186898       1.000000  0.501727   
               gasolina           -0.162000       0.501727  1.000000   
               gasolina aditivada  0.027486       0.261135  0.840504   

                                                      
Produto                           gasolina aditivada  
               Produto                                
Bandeira                                    0.027486  
Valor de Venda etanol                       0.261135  
               gasolina                     0.840504  
               gasolina aditivada           1.000000

In [ ]:
# IDEIAS

# - Pegar as principais bandeiras (que vende todos os combs e esta presente em todas regioes) e plotar
#   um gráfico de barras

# - Fazer igual aquele gráfico de barras azuis que o José fez nos testes dele (cada barra uma bandeira
#   e seus preços médios)
#        - separar por combustivel

In [ ]:
#media_bandeira2 = media_combustivel_bandeira.unstack('Bandeira')
#media_bandeira2

In [ ]:
#correlacao4 = media_bandeira2.corr()
#correlacao4

In [ ]:
sns.heatmap(correlacao3, annot = True, fmt=".1f")